<a href="https://colab.research.google.com/github/Thiago-Fontenelle/Ada_Airlines/blob/main/Projeto_Ada_Airlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simulação de aeroporto

## Imports

In [1]:
import random
import time
from datetime import datetime
from string import ascii_uppercase, digits
import pandas as pd

random.seed(42)

##Funções Geradoras

### air_starter

In [2]:
def air_starter(size: int)-> dict:
  '''
  Recebe número inteiro.
  Retorna dicionários baseados nesse número inteiro que podem ser usados commo pistas, prateleiras e filas.
  '''
  
  new_starter = {i+1: [] for i in range(size)}
  
  return new_starter

### IDs

In [3]:
def id_gen(id_type: int, id_list, len_arg: int = 3)-> str:
  '''
  Recebe número inteiro para a definição do tipo de ID, número inteiro para o tamanho da ID 
  e lista para armazenar IDs já geradas.
  Retorna string única, com len_arg letras e len_arg dígitos, 
  aleatória e que não esteja presente na lista de IDs.
  '''
  if id_type == 0:
    id =  ''.join(random.sample(ascii_uppercase[:13], k=len_arg)+random.sample(digits, k=len_arg))
    while id in id_list:
      id =  ''.join(random.sample(ascii_uppercase[:13], k=len_arg)+random.sample(digits, k=len_arg))
  else:
    id =  ''.join(random.sample(ascii_uppercase[13:], k=len_arg)+random.sample(digits, k=len_arg))
    while id in id_list:
      id =  ''.join(random.sample(ascii_uppercase[13:], k=len_arg)+random.sample(digits, k=len_arg))
  id_list += [id]
  
  return id



###Gerador de IDs - (desenvolvendo)



In [4]:
def plane_generator(plane_type: int, id_list: list, 
                    fuel_min: int = 2, fuel_max: int = 20, cicle_id: str = None) -> dict:
  '''
  Recebe tipo de avião a ser gerado (0= aterriságem 1= decolagem), lista de ids para evitar repetições, 
  nível mínimo e máximo de combistível(inteiros) e id do ciclo(opcional).
  Retorna dicionário com 6 chaves contendo os dados do avição gerado.
  Exemplo:
  {'lane': None,
   'ID': 'HCM895',
   'fuel': 16,
   'wait': 0,
   'cicle': None,
   'final_lane': None}
  '''
  
  # Entrada 0 gera id de aterrissagem
  if plane_type == 0:
    new_id = {'lane': None, 'ID': id_gen(id_type = 0, id_list = id_list), 
              'fuel': random.randint(fuel_min, fuel_max), 'wait': 0, 'cicle': cicle_id, 'final_lane': None}
    
    return new_id
  
  # Entrada 1 gera id de decolagem
  elif plane_type == 1:
    new_id = {'lane': None, 'ID': id_gen(id_type = 1, id_list = id_list), 
              'fuel': random.randint(fuel_min, fuel_max), 'wait': 0, 'cicle': cicle_id, 'final_lane': None}

    return new_id

  else:
    raise ValueError('Value not in accepted range')

###Gerador de aviões

In [5]:
def air_entry(plane_type: int, arrivals: dict, id_list: list, 
              p_min: int = 0, p_max: int = 3, cicle_id: str = None)-> None:
  
  '''
  Recebe o tipo de avião a ser gerado(0 ou 1), dicionário que receberá os aviões gerados,
  lista de IDs para evitar repetições, mínimo e máximo(p_min, p_max) de aviões a serem gerados 
  e ID do ciclo(opcional)
  Altera dicionários que servem como filas e prateleiras para aviões.
  '''

  # plane_type == 0 gera avisões para aterrissaem
  if plane_type == 0:
    arrivals = shelves
    
    for i in arrivals:
      # evitando um 'randint(0, p_max < 0)'
      if len(arrivals[i]) < p_max:
        # evitando que a lotação de fila ultrapasse o limite estabelecido
        p_max -= len(arrivals[i])
        planes = random.randint(p_min, p_max)
        
        if planes > 0:
          for j in range(planes):
            new_plane = plane_generator(plane_type= plane_type, cicle_id = cicle_id, id_list = id_list)
            new_plane['lane'] =  i
            arrivals[i] += [new_plane]
  
  # plane_type == 1 gera aviões para decolagem
  elif plane_type == 1:
    arrivals = t_o_lines
    
    for i in arrivals:
      # evitando um 'randint(0, p_max < 0)'
      if len(arrivals[i]) < p_max:
        # evitando que a lotação de fila ultrapasse o limite estabelecido
        p_max -= len(arrivals[i])
        planes = random.randint(p_min, p_max)
        
        if planes > 0:
          for j in range(planes):
            new_plane = plane_generator(plane_type= plane_type, cicle_id = cicle_id, id_list = id_list)
            new_plane['lane'] = i
            arrivals[i] += [new_plane]
  
  else:
    raise ValueError("No valid 'plane_type' informed")

##Funções de equilíbrio

###Fila de prioridade

In [6]:
def priorities(shel: dict) -> list:
  '''
  Recebe um dicionário de aviões em prateleiras.
  Retorna uma lista de listas, com cada lista aninhada contendo os aviões com nível de combustível baixo.
  Combusível baixo é definido entre prioridade maior ("fuel" == 1) e prioridade menor ("fuel" == 3).
  Aviões dentro das listas de prioridade são definidos ordenados por tempo de espera.
  '''
  prio_1 = sorted([j for i in shel for j in shel[i] if j['fuel'] == 1 
                   if len(shel[i]) > 0], key=lambda prio: prio['wait'], reverse = True)
  
  prio_2 = sorted([j for i in shel for j in shel[i] if j['fuel'] == 2 
                   if len(shel[i]) > 0], key=lambda prio: prio['wait'], reverse = True)

  prio_3 = sorted([j for i in shel for j in shel[i] if j['fuel'] == 3 
                   if len(shel[i]) > 0], key=lambda prio: prio['wait'], reverse = True)

  return [prio_1, prio_2, prio_3]

###prio_line_manager

In [7]:
def prio_line_manager(spaces: dict, air: dict)-> None:
  '''
  Recebe pistas e prateleiras.
  Roda função de prioridades.
  Decide se aviões na lista de prioidades devem ser aterrissados imediatamente.
  '''
  
  prios = priorities(shel= air)

  # Laço 'for' para fazer a checagem filas automaticamente para todas as pistas não populadas nessa rodada
  for p in prios[0]:  
      
    for n in range(len(spaces), 0, -1):
      if len(spaces[n]) == 0 and len(prios[0]) > 0:
          
        fuel_1 = prios[0][0]
        spaces[n] = air[fuel_1['lane']].pop(air[fuel_1['lane']].index(fuel_1))
        spaces[n]['final_lane'] = n
        prios[0].pop(0)
                
    if len(prios[1]) > 0:
      # Criando condição de emergência caso muitos aviões estejam com 2 de combustivel
      if len(prios[1]) < 3:
        final_range = len(spaces) -1
      else:
        final_range = 0
  
      for n in range(len(spaces), final_range, -1):
        if len(spaces[n]) == 0 and len(prios[1]) > 0:
            
          fuel_2 = prios[1][0]
          spaces[n] = air[fuel_2['lane']].pop(air[fuel_2['lane']].index(fuel_2))
          spaces[n]['final_lane'] = n
          prios[1].pop(0)
    # aviões de prioridade 3 têm prioridade menor e usam apenas a pista 3
    if len(spaces[3]) == 0 and len(prios[2]) > 2:
      
      fuel_3 = prios[2][0]
      spaces[3] = air[fuel_3['lane']].pop(air[fuel_3['lane']].index(fuel_3))
      spaces[3]['final_lane'] = 3
      prios[2].pop(0)

###line_manager

In [8]:
def line_manager(spaces: dict, air: dict, ground: dict)-> None:
  '''
  Recebe dicionários de filas e prateleiras.
  Decide quais aviões devem usar as pistas após as prioridades terem sido consideradas.
  Decisão é feita na hierarquia de comparação:
  Maior fila
    Maior tempo de espera
      Se tempo de espera for igual, aviões nas prateleiras tem prioridade.
  '''
  for i in range(1, len(spaces)+1): 
    # Checagem é feita apenas para pistas que ainda não foram populadas nessa rodada
    if len(spaces[i]) == 0:
      if len(air) == len(ground):
        g_idx = i
      else:
        g_idx = 1
      
      size_lanes = [len(air[i]), len(ground[g_idx])]
      
      # Prioridade principal é para o tamanho das filas, assim mantendo o aeroporto rodando com o máximo de aviões
      if sum(size_lanes) > 0:
        
        # Se as filas tiverem o mesmo tamanho, a prioridade passa a ser o tempo de espera
        if size_lanes[0] == size_lanes[1]:
          
          # Se os tempos de espera forem iguais, a prioridade é para aterrisagem por questões de segurança
          if air[i][0]['wait'] >= ground[g_idx][0]['wait']:
            air[i][0]['final_lane'] = i
            spaces[i] = air[i].pop(0)
          else:
            ground[g_idx][0]['final_lane'] = i
            spaces[i] = ground[g_idx].pop(0)
        
        # Condicionais para filas de espera que tenham tamanhos diferentes
        elif size_lanes[0] < size_lanes[1]:
          ground[g_idx][0]['final_lane'] = i
          spaces[i] = ground[g_idx].pop(0)
        elif size_lanes[0] > size_lanes[1]:
          air[i][0]['final_lane'] = i
          spaces[i] = air[i].pop(0)

##Funções de Ciclo

### Line roll

In [9]:
def line_roll(spaces: dict, database: list, air: dict, ground: dict, crash: bool = False) -> None:
  '''
  Recebe a base de dados temporária(database), filas (air, ground) e pistas(spaces).
  percorre as filas reduzindo o combustível dos aviões no ar 
  e aumentando o tempo de espera de todos os aviões.
  No caso de acidente(combustível== 0), registra o acidente, cancela todos os vôos de decolágem e 
  aciona a função clean_up() para evacuar o aeroporto.
  '''
  plane_fall = []
  idx_fall = []

  for j in air:
    # Rotação de todos os aviões nas prateleiras para reduzir combustível e aumentar tempo de espera  
    for k in range(len(air[j])):
    
      air[j][k]['fuel'] -= 1
      air[j][k]['wait'] += 1

      if air[j][k]['fuel'] < 1:
          
        # Registro de acidente aéreo
        crash = True
        plane_fall += [air[j][k]]
        idx_fall += [k]

  if crash == False:      
    for l in ground: 
      for idx in range(len(ground[l])):
        ground[l][idx]['wait'] += 1

  if crash == True:
    for i in range(len(plane_fall)):
      # final_lane: 321 significa acidente, para análises de dados posteriores
      plane_fall[i]['final_lane'] = 321
            
      air_header(text= '!!! ACIDENTE AÉREO !!!')
      display(pd.DataFrame(plane_fall))
      print()
      
      database.append(plane_fall)
      del air[plane_fall[i]['lane']][idx_fall[i]]
    
    # Cancelamento de todos os vôos no chão
    for p in ground:
      canceled = []
      for idx in range(len(ground[p])):
        # final_lane: 0 significa cancelamento, para análises de dados posteriores
        ground[p][0]['final_lane'] = 000
        canceled.append(ground[p][0].copy())
        database.append(ground[p].pop(0))
      
      air_header(text= '!!! VÔOS CANCELADOS !!!')
      display(pd.DataFrame(canceled))
      print()
    
    clean_up(spaces = spaces, database = database, air = air, ground = ground, crash = True)

###clean_up

In [10]:
def clean_up(spaces: dict, database: list, air: dict, ground: dict, crash: bool = False):
  '''
  Recebe a base de dados temporária(database), filas (air, ground) e pistas(spaces).
  Faz o ciclo normal de esvaziamento de filas sem adicionar novos aviões,
  Salva os aviões na base de dados com update_database().
  '''
  larger = []
  larger += [len(ground[1])]

  for i in air:
    larger += [len(air[i])]
  
  for j in range(max(larger)):
    # Como a checagem de acidente é feita durante um line_roll(), essa função não deve ser rodada na
    #   primeira iteração da função clean_up() em caso de acidente
    if crash == False:  
      line_roll(spaces = spaces, database = database, air = air, ground = ground)
    prio_line_manager(spaces = spaces, air = air)
    line_manager(spaces = spaces, air = air, ground = ground)

    crash = False
    
    air_header(f'OVERTIME[{j+1}]')
    update_database(spaces = spaces, database = database)

###update_database

In [11]:
def update_database(spaces: dict, database: list)-> None:  
  '''
  Recebe pistas e base de dados.
  Percorre as pistas, adiciona seu conteúdo à base de dados e limpa as pistas.
  '''
  print()
  display(pd.DataFrame(spaces).transpose())
  
  for j in spaces:
    if len(spaces[j]) > 0:
      database.append(spaces[j].copy())
      spaces[j].clear()
  print()

###air_header

In [12]:
def air_header(text: str, sub_header: bool= False, new_line: bool= True):
  '''
  Recebe string.
  Executa print() formatado para ser o cabeçalho ou sub-cabeçalho de uma tabela.
  '''
  if sub_header == False:
    if new_line == True:
      print()
    print('+'+'-'*59+'+')
    print('|'+text.center(59, ' ')+'|')
    print('+'+'-'*59+'+')
  else:
    if new_line == True:
      print()
    print(('+'+'-'*29+'+').center(59, ' '))
    print(('|'+text.center(29, ' ')+'|').center(59, ' '))
    print(('+'+'-'*29+'+').center(59, ' '))

###database_file

In [13]:
def database_file(database:list, new_file: bool, name: str='summary.csv'):
  '''
  Recebe base de dados e nome do arquivo CSV.
  Se o parametro new_file= True cria novo arquivo, senão, adiciona dados à arquivo selecionado
  '''
  if new_file == True:
    summary = pd.DataFrame(database)
    summary.to_csv(name, index=False)
  
  else:
    new_data = pd.DataFrame(database)
    pd.read_csv(name).append(new_data).drop_duplicates().to_csv(name)

###air_cicle

In [17]:
def air_cicle(spaces: dict, database: list, air: dict, ground: dict, id_list: list, work_time: int = 20, 
              f_min: int = 0, f_max: int = 20, delay:int = 0, unicicle: bool = False, cicle = None, new_file: bool= True):
  '''
  Recebe todas as variáveis de funcionamento: base de dados, filas(decolagem, aterriságem), lista de IDs,
  combustível(máximo e mínimo), time.sleep(opcional), unicicle e cicle.
  Roda todas as funções em sequência de forma automática em um número de repetições definido por work_time
  e limpa os aviões remanescentes após o fim das repetições.
  No caso de unicicle= True recomenda-se o preenchimento de cicle com um datetime único para 
  gerar a ID do ciclo e, após teminar suas repetições manuais desejadas, rodar a função com unicicle= False
  e work_time= 0 para a limpeza dos aviões remanescentes.
  '''
  if cicle == None:
    cicle_id = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
  

  for i in range(work_time):

    round = i+1
    
    # Atualizar tempo
    line_roll(spaces= spaces, database= database, air= air, ground= ground)
    
    # Adicionar aviões
    air_entry(plane_type= 0, arrivals= air, cicle_id= cicle_id, id_list= id_list)
    air_entry(plane_type= 1, arrivals= ground, cicle_id= cicle_id, id_list= id_list)
    
    # Administrar filas
    prio_line_manager(spaces= spaces, air = air)
    line_manager(spaces= spaces, air= air, ground= ground)
    
    # Mostrar na tela dados sobre o ciclo
    air_header(text= f'STATUS CICLE[{i+1}]')
    
    # Sub-tabela de status de prateleiras
    status_shelves = [{'shelf': i, 'total_planes': len(air[i]), 'IDs': [j['ID'] for j in air[i]]} for i in range(1, len(air)+1)]
    air_header(text= 'STATUS SHELVES', sub_header= True)
    display(pd.DataFrame(status_shelves))
    
    # Sub-tabela de status de filas
    status_lines = [{'line': i, 'total_planes': len(ground[i]), 'IDs': [j['ID'] for j in ground[i]]} for i in range(1, len(ground)+1)]
    air_header(text= 'STATUS LINES', sub_header= True)
    display(pd.DataFrame(status_lines))
    
    # Sub-tabela de status de espera
    landing_wait = [i['wait'] for i in database if i['ID'][0] in ascii_uppercase[:13]]
    take_off_wait = [i['wait'] for i in database if i['ID'][0] in ascii_uppercase[13:]]
    if len(landing_wait) > 0 and len(take_off_wait) > 0:
      med_lan = sum(landing_wait)/len(landing_wait)
      med_t_o = sum(take_off_wait)/len(take_off_wait)
    else: 
      med_lan = 0
      med_t_o = 0
      
    emergency_lan = sum([1 for p in database if p['fuel']==1])
    
    wait_status = [{'landing_wait': med_lan,
                    'take_off_wait': med_t_o,
                    'emergency_landings': emergency_lan}]
    
    air_header(text= 'STATUS WAIT', sub_header= True)
    display(pd.DataFrame(wait_status))
    
    # Carregar aviões que já chegaram nas lanes para base de dados
    air_header(text= "LANES", sub_header= True)
    print()
    update_database(spaces= spaces, database= database)
    
    time.sleep(delay)
  
  # Limpar aviões remanescentes nas filas após o fim do ciclo
  if unicicle == False:

    time.sleep(delay)
    # Limpeza de aviões remanescentes
    clean_up(spaces= spaces, database= database, air= air, ground= ground)
    
    # Tabela de resultados finais
    result = pd.DataFrame(database)
    air_header(text= 'FINAL RESULT')
    display(result)
    
    emergency_lan = sum([1 for p in database if p['fuel']==1])
    report = [{'total planes': len(database), 
                'medium_wait': sum([i['wait'] for i in database])/len(database),
                'crashes': len([1 for i in range(len(database)) if database[i]['final_lane']==321]),
                'canceled': len([1 for i in range(len(database)) if database[i]['final_lane']==0]),
                'emergency_landings': emergency_lan}]
    report = pd.DataFrame(report).transpose()
    report = report.round(decimals= 2)
    
    # Tabela de metadados finais
    air_header(text= 'RESULT REPORT')
    display(report)

    # Gerar CSV
    database_file(database= database, new_file= new_file, name= 'summary.csv')

##Base de dados fabricada para testes

In [ ]:
test_total_planes = []

In [ ]:
test_lanes = {1 : [], 2 : [], 3 : []}

In [ ]:
test_shelves = {1: [{'lane': 1, 'ID': 1, 'fuel': 10, 'wait': 0, 'time': 0, 'final_lane': None}, 
                    {'lane': 1, 'ID': 11, 'fuel': 9, 'wait': 0, 'time': 0, 'final_lane': None}, 
                    {'lane': 1, 'ID': 13, 'fuel': 3, 'wait': 0, 'time': 0, 'final_lane': None}], 
                2: [{'lane': 2, 'ID': 3, 'fuel': 2, 'wait': 0, 'time': 0, 'final_lane': None}, 
                    {'lane': 2, 'ID': 5, 'fuel': 1, 'wait': 0, 'time': 0, 'final_lane': None}, 
                    {'lane': 2, 'ID': 7, 'fuel': 14, 'wait': 0, 'time': 0, 'final_lane': None}], 
                3: [{'lane': 3, 'ID': 9, 'fuel': 3, 'wait': 0, 'time': 0, 'final_lane': None}, 
                    {'lane': 3, 'ID': 15, 'fuel': 16, 'wait': 0, 'time': 0, 'final_lane': None}, 
                    {'lane': 3, 'ID': 17, 'fuel': 2, 'wait': 0, 'time': 0, 'final_lane': None}]}

test_t_o_lines = {1: [{'lane': 1, 'ID': 2, 'fuel': 20, 'wait': 0, 'time': 0, 'final_lane': None}, 
                      {'lane': 1, 'ID': 4, 'fuel': 20, 'wait': 0, 'time': 0, 'final_lane': None}, 
                      {'lane': 1, 'ID': 6, 'fuel': 20, 'wait': 0, 'time': 0, 'final_lane': None}]}

In [ ]:
test_id_list = []

In [ ]:
line_roll(spaces = test_lanes, database = test_total_planes, air = test_shelves, 
              ground = test_t_o_lines, crash = False)


+-----------------------------------------------------------+
|                   !!! ACIDENTE AÉREO !!!                  |
+-----------------------------------------------------------+


,lane,ID,fuel,wait,time,final_lane
0,2,5,0,1,0,321




+-----------------------------------------------------------+
|                   !!! ACIDENTE AÉREO !!!                  |
+-----------------------------------------------------------+


,lane,ID,fuel,wait,time,final_lane
0,1,2,20,0,0,0
1,1,4,20,0,0,0
2,1,6,20,0,0,0




+-----------------------------------------------------------+
|                        OVERTIME[1]                        |
+-----------------------------------------------------------+



,lane,ID,fuel,wait,time,final_lane
1,1,1,9,1,0,1
2,3,17,1,1,0,2
3,2,3,1,1,0,3




+-----------------------------------------------------------+
|                        OVERTIME[2]                        |
+-----------------------------------------------------------+



,lane,ID,fuel,wait,time,final_lane
1,1,11,7,2,0,1
2,3,9,1,2,0,2
3,1,13,1,2,0,3




+-----------------------------------------------------------+
|                        OVERTIME[3]                        |
+-----------------------------------------------------------+



,lane,ID,fuel,wait,time,final_lane
1,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,7.0,11.0,3.0,0.0,2.0
3,3.0,15.0,13.0,3.0,0.0,3.0


In [ ]:
prio_line_manager(spaces = test_lanes, air = test_shelves)

In [ ]:
line_manager(spaces = test_lanes, air = test_shelves, ground = test_t_o_lines)

In [ ]:
print(test_lanes)
print(len(test_shelves[1]), len(test_shelves[2]), len(test_shelves[3]))
print(len(test_lanes[1]), len(test_lanes[2]), len(test_lanes[3]))
print(len(test_total_planes))
test_total_planes

{1: {}, 2: {}, 3: {}}
0 0 0
0 0 0
12


[[{'lane': 2, 'ID': 5, 'fuel': 0, 'wait': 1, 'time': 0, 'final_lane': 321}],
 {'lane': 1, 'ID': 2, 'fuel': 20, 'wait': 0, 'time': 0, 'final_lane': 0},
 {'lane': 1, 'ID': 4, 'fuel': 20, 'wait': 0, 'time': 0, 'final_lane': 0},
 {'lane': 1, 'ID': 6, 'fuel': 20, 'wait': 0, 'time': 0, 'final_lane': 0},
 {'lane': 1, 'ID': 1, 'fuel': 9, 'wait': 1, 'time': 0, 'final_lane': 1},
 {'lane': 3, 'ID': 17, 'fuel': 1, 'wait': 1, 'time': 0, 'final_lane': 2},
 {'lane': 2, 'ID': 3, 'fuel': 1, 'wait': 1, 'time': 0, 'final_lane': 3},
 {'lane': 1, 'ID': 11, 'fuel': 7, 'wait': 2, 'time': 0, 'final_lane': 1},
 {'lane': 3, 'ID': 9, 'fuel': 1, 'wait': 2, 'time': 0, 'final_lane': 2},
 {'lane': 1, 'ID': 13, 'fuel': 1, 'wait': 2, 'time': 0, 'final_lane': 3},
 {'lane': 2, 'ID': 7, 'fuel': 11, 'wait': 3, 'time': 0, 'final_lane': 2},
 {'lane': 3, 'ID': 15, 'fuel': 13, 'wait': 3, 'time': 0, 'final_lane': 3}]

#Execução

In [18]:
# Pistas (decolagem/aterrissagem) onde os aviões serão armazenados temporariamente
lanes = air_starter(size = 3)
# Prateleiras (aterrisagem)
shelves = air_starter(size = 3)
# Filas de decolagem
t_o_lines = air_starter(size = 1)
# Lista de IDs para evitar repetição
id_list = []
# Base de dados de aviões que passaram pelo aeroporto a ser populada e exportada
total_planes = []

In [19]:
air_cicle(spaces = lanes, database = total_planes, air = shelves, ground = t_o_lines, 
          id_list = id_list, work_time= 20, new_file= True)


+-----------------------------------------------------------+
|                      STATUS CICLE[1]                      |
+-----------------------------------------------------------+

              +-----------------------------+              
              |        STATUS SHELVES       |              
              +-----------------------------+              


,shelf,total_planes,IDs
0,1,0,[]
1,2,1,[DFB169]
2,3,1,[BIE953]



              +-----------------------------+              
              |         STATUS LINES        |              
              +-----------------------------+              


,line,total_planes,IDs
0,1,0,[]



              +-----------------------------+              
              |         STATUS WAIT         |              
              +-----------------------------+              


...

,lane,ID,fuel,wait,cicle,final_lane
1,1,KDI721,10,1,31/10/2022 23:13:59,1
2,2,BDK438,5,1,31/10/2022 23:13:59,2
3,NaN,NaN,NaN,NaN,NaN,NaN




+-----------------------------------------------------------+
|                        FINAL RESULT                       |
+-----------------------------------------------------------+


,lane,ID,fuel,wait,cicle,final_lane
0,1,LKI637,20,0,31/10/2022 23:13:59,1
1,2,MAC654,6,0,31/10/2022 23:13:59,2
2,3,JEA781,14,0,31/10/2022 23:13:59,3
3,1,MEB316,10,0,31/10/2022 23:13:59,1
4,2,DFB169,12,1,31/10/2022 23:13:59,2
...,...,...,...,...,...,...
57,1,KFJ514,16,2,31/10/2022 23:13:59,1
58,2,KGF642,6,2,31/10/2022 23:13:59,2
59,1,RQT957,16,1,31/10/2022 23:13:59,3
60,1,KDI721,10,1,31/10/2022 23:13:59,1



+-----------------------------------------------------------+
|                       RESULT REPORT                       |
+-----------------------------------------------------------+


,0
total planes,62.00
medium_wait,1.06
crashes,0.00
canceled,0.00
emergency_landings,3.00
